# Catalog Comparisons

In [1]:
import pandas as pd
import geopandas as gpd
import rioxarray as rxr

from gval.catalogs.catalogs import catalog_compare

## Initializing Catalogs

The cataloging functionality was designed to easily facilitate batch comparisons of maps residing locally, in a service, or in the cloud.  The format of such catalogs are as follows:

In [2]:
TEST_DATA_DIR = './'

candidate_catalogs = [pd.read_csv(f'{TEST_DATA_DIR}candidate_catalog_{x}.csv') for x in range(2)]
benchmark_catalogs = [pd.read_csv(f'{TEST_DATA_DIR}benchmark_catalog_{x}.csv') for x in range(2)]

In [3]:
candidate_catalogs[1]

,Unnamed: 0,map_id,compare_id,value1,value2,agreement_maps
0,0,./candidate_categorical_0.tif,compare1,1,5,agreement_categorical_0.tif
1,1,./candidate_categorical_1.tif,compare2,2,6,agreement_categorical_1.tif


In [4]:
on = ["compare_id"] * 2
map_ids = ["map_id", "map_id"]
how = ["inner"] * 2
compare_type = ["continuous"] + ["categorical"] 
dask = [False, False]

compare_kwargs = [
    {
        "metrics": (
            "coefficient_of_determination",
            "mean_absolute_error",
            "mean_absolute_percentage_error",
        ),
        "encode_nodata": True,
        "nodata": -9999,
    },
    {
        "metrics": (
            "critical_success_index",
            "true_positive_rate",
            "positive_predictive_value",
        ),
        "encode_nodata": True,
        "nodata": -9999,
        "positive_categories": 2,
    }
]

open_kwargs = [{"mask_and_scale": True, "masked": True}] * 4
agreement_map_field = ["agreement_maps"] * 4

In [39]:
agreement_catalog = catalog_compare(
        candidate_catalogs,
        benchmark_catalogs,
        on=on,
        map_ids=map_ids,
        how=how,
        compare_type=compare_type,
        compare_kwargs=compare_kwargs,
        open_kwargs=open_kwargs,
        agreement_map_field=agreement_map_field,
    )

AttributeError: 'list' object has no attribute 'merge'